In [2]:
#export
import nbtex.core.operators as Operator
from nbtex.helpers import varArgFunc
from nbtex.core.Precedence import PRECEDENCE
from nbtex.LatexInterface import LatexBasicFormatter

class Var:
    def __init__(self, name, backward_operation=None):
        self._name, self._backward_operation = name, backward_operation
    
    @property
    def backward_operation(self):
        return self._backward_operation
    
    @property
    def precedence(self):
        return (PRECEDENCE.VAR 
                if self._backward_operation is None 
                else self._backward_operation.precedence)
    
    def build(self):
        return (self._name 
                if self._backward_operation is None 
                else self._backward_operation.build())
    
    @staticmethod
    def operation(operator, *args):
        operation = Operation(operator)
        args = makeVar(*args)
        if isinstance(args, list):
            output = Var('output', operation(*args))
        else:
            output = Var('output', operation(args))
        return output
    
    def __invert__(self):
        if self._backward_operation is not None:
            self._backward_operation = self._backward_operation.__invert__()
        return self
        
        
    def __call__(self, *args):
        return Var.operation(Operator.FunctionCall, self, *args)
    
    def __add__(self, other):
        return Var.operation(Operator.Plus, self, other)
    def __sub__(self, other):
        return Var.operation(Operator.Minus, self, other)
    def __mul__(self, other):
        return Var.operation(Operator.Multiply, self, other)
    def __pow__(self, other):
        return Var.operation(Operator.Power, self, other)
    def __or__(self, other):
        return Var.operation(Operator.Space, self, other)
    
    def __lt__(self, other):
        return Var.operation(Operator.LessThan, self, other)
    def __gt__(self, other):
        return Var.operation(Operator.GreaterThan, self, other)    
    def __le__(self, other):
        return Var.operation(Operator.LessThanEqual, self, other)
    def __ge__(self, other):
        return Var.operation(Operator.GreaterThanEqual, self, other)
    def __eq__(self,other):
        return Var.operation(Operator.Equal, self, other)
    def __ne__(self,other):
        return Var.operation(~Operator.Equal, self, other)
    def __neg__(self):
        return Var.operation(Operator.Negate, self)
    def __truediv__(self, other):
        return Var.operation(Operator.Fraction, self, other)
    
    def root(self, other):
        return Var.operation(Operator.Root, self, other)
    
    def sum_over(self, wrt='', lower='', upper=''):
        return Var.operation(Operator.Summation, self, wrt, lower, upper)
    def product_over(self, wrt='', lower='', upper=''):
        return Var.operation(Operator.Product, self, wrt, lower, upper)
    
    def integral_over(self, wrt='', lower='', upper=''):
        return Var.operation(Operator.Integral, self, wrt, lower, upper)
    def differentiate(self, wrt):
        return Var.operation(Operator.Derivative, self, wrt)
    
    def partial_integral_over(self, wrt='', lower='', upper=''):
        return Var.operation(Operator.PartialIntegral, self, wrt, lower, upper)
    def partial_differentiate(self, wrt):
        return Var.operation(Operator.PartialDerivative, self, wrt)
    
    
def isVar(a):
    return isinstance(a, Var)

def makeVar(*args):
    return varArgFunc(lambda arg: arg if isVar(arg) else Var(str(arg)), *args)

def addEnv(s, env='gather*'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env

In [3]:
#export
class Operation:
    def __init__(self, operator):
        self._operator = operator
        self._args = None
    
    @property
    def precedence(self):
        return self._operator.precedence
    
    def __call__(self, *args):
        self._args = args
        return self
    
    def _is_invertible(self, op):
        return isinstance(op, Operator.InvertibleOperator)
    
    def build(self):
        args = [self._build_and_surround(makeVar(arg)) for arg in self._args]
        return self._operator(*args)
    
    def _build_and_surround(self, var_instance):
        return (self._get_surrounded_with_parens(var_instance.build())
               if self._should_surround_with_parens(var_instance)
               else var_instance.build())
    
    def _get_surrounded_with_parens(self, string):
        return LatexBasicFormatter.surround_with_parens(string)
    
    def _should_surround_with_parens(self, var_instance):
        return (var_instance.backward_operation is not None and 
                var_instance.backward_operation.precedence < self.precedence)   
    
    def __invert__(self):
        if self._is_invertible(self._operator):
            self._operator = ~self._operator
        return self

In [4]:
from nbtex.platforms.jupyter import latex
a, b = makeVar('a', 'b')
latex(~(a.root(b)))

<IPython.core.display.Latex object>

In [5]:
latex(a.root(b).differentiate('x') + -b)

<IPython.core.display.Latex object>

In [6]:
f, x = makeVar('f', 'x')
latex(f(x).partial_integral_over('i', '1', 'n'))

<IPython.core.display.Latex object>

In [7]:
latex(~a(b))

<IPython.core.display.Latex object>

In [8]:
latex(a.partial_differentiate('x'))

<IPython.core.display.Latex object>

In [11]:
! ../notebook2script.py Ast.ipynb

Traceback (most recent call last):
  File "../notebook2script.py", line 3, in <module>
    import json,fire,re
ModuleNotFoundError: No module named 'fire'
